In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import re
import gc
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load the data
df_train = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/cleaned/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/cleaned/test.csv")
df_val = pd.read_csv("/content/drive/My Drive/fnd-usingML/dataset/cleaned/val.csv")

In [ ]:
for file in range(0,len(df_train)):
  f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/train/text{}.txt'.format(file), 'w')
  temp = df_train['text'][file]
  f.write(temp)
  f.close()

In [ ]:
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/train/labels.txt", 'w')
for i in range(0,len(df_train['isfake'])):
  temp = str(df_train['isfake'][i]) + '\n'
  f.write(temp) 
f.close()  

In [ ]:
for file in range(0,len(df_test)):
  f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/test/text{}.txt'.format(file), 'w')
  temp = df_test['text'][file]
  f.write(temp)
  f.close()

In [ ]:
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/test_label/labels.txt", 'w')
for i in range(0,len(df_test['isfake'])):
  temp = str(df_test['isfake'][i]) + '\n'
  f.write(temp) 
f.close()  

In [5]:
for file in range(0,len(df_val)):
  f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/val/text{}.txt'.format(file), 'w')
  temp = df_val['text'][file]
  f.write(temp)
  f.close()

In [ ]:
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/val_label/labels.txt", 'w')
for i in range(0,len(df_val['isfake'])):
  temp = str(df_val['isfake'][i]) + '\n'
  f.write(temp) 
f.close()  

In [9]:
# Obtain additional stopwords from nltk
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [11]:
df_train['clean'] = df_train['text'].apply(preprocess) 

In [12]:
# Obtain the total words present in the dataset
list_of_words = []
for i in df_train.clean:
    for j in i:
        list_of_words.append(j)

In [13]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

183619

In [14]:
# join the words into a string
df_train['clean_joined'] = df_train['clean'].apply(lambda x: " ".join(x))

In [15]:
# length of maximum document will be needed to create word embeddings 
maxlen = -1
for doc in df_train.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)

In [16]:
x_train = df_train['clean_joined']
y_train = df_train['isfake']

In [17]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)

In [ ]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = maxlen, padding = 'post', truncating = 'post') 

In [18]:
df_train

,text,isfake,clean,clean_joined
0,TOKYO (Reuters) - Japan is to acquire medium-r...,0,"[tokyo, reuters, japan, acquire, medium, range...",tokyo reuters japan acquire medium range launc...
1,(CNN) Hillary Clinton might be on the way to t...,0,"[hillary, clinton, democratic, presidential, n...",hillary clinton democratic presidential nomina...
2,Obama Criticizes McCain on Lobbyists,0,"[obama, criticizes, mccain, lobbyists]",obama criticizes mccain lobbyists
3,Remember back in May when Donald Trump said he...,1,"[remember, donald, trump, said, california, yo...",remember donald trump said california york mar...
4,Military: Goal Is to 'Liberate' Eastern Bank o...,1,"[military, goal, liberate, eastern, bank, tigr...",military goal liberate eastern bank tigris riv...
...,...,...,...,...
67503,"By Julian Assange \nIn recent months, WikiLeak...",1,"[julian, assange, recent, months, wikileaks, p...",julian assange recent months wikileaks persona...
67504,Testimony on CBO’s Assessment of the Air Force...,0,"[testimony, assessment, force, plan, acquire, ...",testimony assessment force plan acquire boeing...
67505,"WEST PALM BEACH, Fla. — Venus Williams’s ab...",0,"[west, palm, beach, venus, williams, ability, ...",west palm beach venus williams ability tennis ...
67506,Sofia Richie and Kourtney Kardashian post comp...,1,"[sofia, richie, kourtney, kardashian, post, co...",sofia richie kourtney kardashian post competin...
